gensim.summarization.summarizer.summarize을 사용하여 cleaned_description 요약 (word_count=70) 및 cleaning, 댓글 합쳐서 요약 (word_count=35)

In [1]:
import numpy as np
import pandas as pd
import re
import pickle
import ast

from gensim.summarization.summarizer import summarize

In [2]:
file_path = 'youtube_0816_0822/add_comment_sentiment_data/cs_{}_0816_0822.csv'

In [3]:
channel_a = pd.read_csv(file_path.format('channel_a'))
jtbc = pd.read_csv(file_path.format('jtbc'))
kbs = pd.read_csv(file_path.format('kbs'))
mbc = pd.read_csv(file_path.format('mbc'))
mbn = pd.read_csv(file_path.format('mbn'))
sbs = pd.read_csv(file_path.format('sbs'))
yonhab = pd.read_csv(file_path.format('yonhab'))
ytn = pd.read_csv(file_path.format('ytn'))

In [4]:
channel_a.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
jtbc.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
kbs.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
mbc.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
mbn.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
sbs.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
yonhab.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)
ytn.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1, inplace=True)

In [5]:
# cleaning에서 미처 삭제하지 못한 이메일
# 특수문자
# 뉴스 진행자, 프로그램 이름 등 뉴스 내용과 관련이 없는 부분

useless_string_list = [r'\n',
                        r'\[\b([a-z0-9가-힣]+(?:[.@]?[a-z]+)+)\b\]',
                        r'\w{2,3}\s?\[[a-z0-9가-힣]+@ytn\.co\.',
                        r'\[앵커\]',
                        r'\[기자\] 네\,',
                        r'\[기자\]',
                        r'\[캐스터\] 네\,',
                        r'\[캐스터\]',
                        r'\[리포터\]',
                        r'\[인터뷰\]',
                        r'\[기사내용\]',
                        r'\[리포트\]',
                        r'\[굿모닝 MBN\]',
                        r'\[굿모닝월드\]',
                        r'스탠딩 \:',
                        r'\[전국네트워크\]',
                        r'\[영상톡\]',
                        r'\[아침\&매일경제\]',
                        r'\[날씨\]',
                        r'\[날씨트리\]',
                        r'\[뉴스리뷰\]',
                        r'\[UHD 다큐 풍경\]',
                        r'\[현장연결\]',
                        r'\[현장음\]',
                        r'\[자막뉴스\]',
                        r'\[칭찬마이크\]',
                        r'\[단독\]',
                        r'\[AM\-PM\]',
                        r'■',
                        r'▶',
                        r'◆'
]

# 기사 본문 요약

In [6]:
def summarize_article(DataFrame, word_count, useless_list):
    '''gensim summarize로 article 요약 및 필요없는 문자열 삭제
    
    DataFrame : cleaned_description이 있는 데이터프레임
    
    word_count : gensim summarize로 요약할때의 단어 수 (요약문 분량)
    
    useless_list : 삭제할 문자열 (또는 정규표현식) 리스트
    '''
    
    copy = DataFrame.copy()
    summary_list = []
    for element in copy.cleaned_description:
        # 불필요한 문자열 제거
        for token in useless_list:
            element = re.sub(token, '', element)
        
        # 기사 요약문 생성
        try:
            summary = summarize(element, word_count=word_count)
            summary = re.sub(r'\n', ' ', summary)
            if summary == '':
                summary = None
                
        except:
            summary = None
        
        summary_list.append(summary)
    
    copy['article_summary'] = summary_list
    copy.dropna(axis=0, inplace=True)
    copy.reset_index(drop=True, inplace=True)
    
    return copy

In [7]:
summary_channel_a = summarize_article(DataFrame=channel_a,
                                      word_count=70,
                                      useless_list=useless_string_list)

In [8]:
summary_jtbc = summarize_article(DataFrame=jtbc,
                                 word_count=70,
                                 useless_list=useless_string_list)

In [9]:
summary_kbs = summarize_article(DataFrame=kbs,
                                word_count=70,
                                useless_list=useless_string_list)

In [10]:
summary_mbc = summarize_article(DataFrame=mbc,
                                word_count=70,
                                useless_list=useless_string_list)

In [11]:
summary_mbn = summarize_article(DataFrame=mbn,
                                word_count=70,
                                useless_list=useless_string_list)

In [12]:
summary_sbs = summarize_article(DataFrame=sbs,
                                word_count=70,
                                useless_list=useless_string_list)

In [13]:
summary_yonhab = summarize_article(DataFrame=yonhab,
                                   word_count=70,
                                   useless_list=useless_string_list)

In [14]:
summary_ytn = summarize_article(DataFrame=ytn,
                                word_count=70,
                                useless_list=useless_string_list)

# 댓글 요약

In [15]:
def summarize_comments(DataFrame, word_count):
    '''댓글 요약
    
    DataFrame : comments가 있는 데이터 프레임
    
    word_count : 댓글을 요약할때의 단어 수 (댓글 요약 분량)
    '''
    
    copy = DataFrame.copy()
    summary_list = []
    fail_case = '댓글 요약 실패!! 영상에서 직접 확인하세요.'
    
    for element in copy.comments:
        try:
            # 문자열로 되어있는 리스트를 리스트로 변환 후 댓글들을 하나의 문자열로 합치기
            element = ' '.join(ast.literal_eval(element))
            # 이모티콘 제거
            element = element.encode('euc-kr', 'ignore').decode('euc-kr')
            element = summarize(re.sub(r'\n', ' ', element), word_count=word_count)
            element = re.sub(r'\n', ' ', element)
            
            # 요약 실패 경우 처리
            if element == '':
                element = fail_case
        
        except:
            element = fail_case
            
        summary_list.append(element)
        
    copy['comments_summary'] = summary_list
    
    return copy

In [16]:
summary_channel_a_1 = summarize_comments(DataFrame=summary_channel_a,
                                         word_count=30)

In [17]:
summary_jtbc_1 = summarize_comments(DataFrame=summary_jtbc,
                                    word_count=30)

In [18]:
summary_kbs_1 = summarize_comments(DataFrame=summary_kbs,
                                   word_count=30)

In [19]:
summary_mbc_1 = summarize_comments(DataFrame=summary_mbc,
                                   word_count=30)

In [20]:
summary_mbn_1 = summarize_comments(DataFrame=summary_mbn,
                                   word_count=30)

In [21]:
summary_sbs_1 = summarize_comments(DataFrame=summary_sbs,
                                   word_count=30)

In [22]:
summary_yonhab_1 = summarize_comments(DataFrame=summary_yonhab,
                                      word_count=30)

In [23]:
summary_ytn_1 = summarize_comments(DataFrame=summary_ytn,
                                   word_count=30)

# 결과 확인용

In [24]:
i = 0
for element in summary_yonhab_1.article_summary:
    print(i)
    print(element)
    print('\n')
    i += 1
    if i > 25:
        break

0
수도권 코로나19 비상 서울, 경기 거리두기 2단계  코로나19 확산세가 크게 늘어 그제 신규 확진자 수가 다섯달 만에 가장 많은 166명에 달했는데요. 방역당국과 지방자치단체의 자체 집계에 따르면 어제 0시 이후 서울과 경기에서 새로 확진 판정을 받은 감염자만, 150명을 넘었습니다. 여기에 해외유입 환자도 더해지면 확진자 수는 더 늘어날 것으로 보입니다. 정부가 확산세가 심각한 서울시와 경기도의 사회적 거리두기를 1단계에서 2단계로 높이기로 했죠? 오늘부터 2주간 서울과 경기도에서는 사회적 거리두기가 2단계로 격상됩니다.


1
엄중 경고에도 자가격리 위반 천태만상  방역당국은 코로나19 확산세가 심상치 않다며 연일 방역수칙을 지켜달라고 당부하고 있죠. 코로나19 확산세가 다시 가팔라지면서 방역당국은 지금을 위기 라고 진단했습니다. 이런 때일수록 기본 방역수칙을 지키는 게 중요한데, 자가격리 위반사례가 계속 나오고 있습니다. 미국에 머물던 20대 정 모 씨는 자가격리 기간 다시 미국으로 출국했다가 덜미를 잡혔습니다. 경기도 의정부에서는 자가격리 무단이탈자에게 처음으로 실형이 확정됐습니다. 정부와 수사기관은 국가적 재난 상황을 감안해 앞으로도 자가격리 위반자는 엄정조치할 계획이라고 밝혔습니다.


2
현장 먼저 찾고 중도로 좌클릭 통합당 변신 몸부림  미래통합당이 최근 과거와는 달라진 면모를 보여주기 위해 몸부림을 치고 있습니다. 그동안 거리가 있었던 호남과 중도층을 끌어안기 위한 시도를 본격화하는 동시에 적극적인 현장 행보를 이어가고 있습니다. [주호영 / 미래통합당 원내대표] 어디까지 보내야 되나 이거. [주호영 / 미래통합당 원내대표] 이거 수레에 빨리 실어야 되는데 통합당 의원들과 당원 300명도 전북 남원 수해 복구 현장에서 일손을 보탰습니다. 한 박자 빠른 통합당의 현장 행보는 이미지 변신을 꾀하려는 상징적 시도로 볼 수 있습니다.


3
서울 신규 확진 80명 사랑제일교회 확진자 급증  서울에서 코로나19 신규 확진자가 어제만 80명이 발생했

In [25]:
i = 0
for element in summary_ytn_1.article_summary:
    print(i)
    print(element)
    print('\n')
    i += 1
    if i > 25:
        break

0
더불어민주당은 오는 29일 전당대회를 앞두고 오늘 호남과 충청권 합동연설회를 열고 미래통합당은 쇄신 작업 의 상징이 될 새 당명을 찾고 있습니다. 민주당은 다시 전당대회 일정을 재개한다고요? 복구 작업이 여전히 진행 중인 데다, 코로나19까지 다시 확산세를 보이자 비대면 방식으로 전당대회 일정을 이어가는 겁니다. 이 같은 쇄신 작업의 상징이 될 새 당명을 지난 13일부터 공모하고 있습니다. 내일 새벽 공개될 YTN 여론조사가 이런 분위기에 또 어떤 식으로 영향을 미칠지 정치권 관심이 더욱 모이고 있습니다.


1
수도권을 중심으로 코로나19 확진자가 빠른 속도로 늘고 있습니다. 오늘 0시 기준 신규 확진자는 모두 279명입니다. 코로나19 확산 세가 수도권을 중심으로 이어지고 있는데, 교회와 상가, 학교 등 곳곳에서 동시다발적으로 감염자가 확인되고 있습니다. 수도권을 중심으로 집단감염이 계속되면서 코로나19 재유행 우려가 현실이 되고 있습니다. 국내 발생 상황을 지역별로 보면 서울에서만 141명이 발생했고, 경기가 96명, 인천이 8명입니다. 방역 당국은 무증상과 경증 환자가 늘어나면서 지역 사회 안에서 조용한 전파 를 일으키는 상황을 주시하고 있습니다.


2
국내외 40여 개의 가구업체와 인테리어 업체가 참가해 새로운 트렌드를 소개하고 올 가을 이사철을 앞두고 신상품을 선보이고 있습니다. [김태희 / 전시 참가업체 직원 : 1인 가구가 급증하는 시기를 맞아 가성비가 좋고 합리적인 상품들을 전시회에 소개하기 위해 참가했습니다.] 코로나19 사태로 오랫동안 집안에서 지내야 했던 중장년층들이 대거 전시회장을 찾아 전시품을 꼼꼼히 챙겨보고 있습니다. 반면 참가업체들의 만족도가 대단히 높은데 그 이유는 진성 고객들이 참관하기 있기 때문입니다.] 컨벤션 산업은 업체 참가자와 관람객 외에 각종 행사 준비업체와 용역업체 등이 참가해 생산 유발 효과가 크지만 코로나19 사태로 올 상반기 최악의 시기를 보낸 후 이제 본격 회복세로 돌아섰습니다.


3
전광훈 목사가 이끄

In [26]:
i = 0
for element in summary_yonhab_1.comments_summary:
    print(i)
    print(element)
    print('\n')
    i += 1
    if i > 25:
        break

0
진보는 코로나 막으려고 애쓰고... 보수는 코로나 퍼트릴려고 애쓰고... 왜?갑자기 늘어났죠? 이시국에.집회 강행 빤스 코로나 민폐충들  때문에 한국도 2차 웨이브 발생하는구나 코로나 수칙 안지킨 인간들은 죽게 생겨도 살려달란 소리 하지말길 갑자기 왜 늘어나냐, 지금까지는  숨겼냐,, 또 교회만  방송하냐,, 지금까지코로나조작했다는증거!


1
자가격리 위반 와 미친 엄중경고에도 안지끼는놈들은 해왜놈은 벌금5억때리고  추방하고  우리나라사람은 감옥 독방에 쳐넉코 벌금5억때리면 대지 참나 그냥 자가 격리자 얼굴 이름 동선  싹 공개해라 저놈들 자업자득이다 자가 격리 어기는 사람들  모두 얼굴 공개하면 된다  그러면 안지킬수가 없다 법이  약해   그러니  않 지키지  강력하게  처벌해야   자가격리  잘 지키지 벌금 대폭 올리고!


2
호박에 줄 그은다고 수박 되지 않는데. 호박에 줄그면.. :  쑈는 기가 막히게 잘해요. 기가 막히게 ??? : 그러게 말입니다. 그러게 진작에 중도파 받아들이지. 그리고 보수는 폐쇄가 아니라 좀 더 열려야 한다.


3
자영업자 영업 못하게 하자는 게 아닙니다. 코로나 잡을려면  차라리 2~3주 멈추는 편이 낮지요....자영업자 지원 좀 해주면서...코로나 질 질 끌면서 정치에 이용하지 말고!!! 가짜 뉴스 퍼트리지마라 진짜ㅡㅡ 수도권 코로나 조작인거 다안다.


4
무슨 찬반이냐 공매도를 반대하면 모든 금융거래기법을 반대하는 거다 개미들도 공부를 많이해서 전문가가 되어야죠 모든 투자자가 같은 선상에서 공매도는 찬성입니다. 이번 개미는 정권을 바꿀 힘이 있습니다.


5
척살했어야.하는데 못한게.지금.이상황을.만든것이다 정치꾼 기회주의자  김원웅이 광복회장이라니 모조리 미쳐 가나보다. 과거 이력을 보니 현란하든데 이나라가 코로나에 대홍수에 어렵고 힘든 시기에 온국민이 힘을 합쳐도 시원챦을 판에 찬일파 묘를 파야하네 마네 이런 일로 소모전을 일으키다니  이런 인간들이 문제!


6
댓글 요약 실패!! 영상에서 직접 확인하세요

In [27]:
i = 0
for element in summary_ytn_1.comments_summary:
    print(i)
    print(element)
    print('\n')
    i += 1
    if i > 25:
        break

0
(1) 수구꼴통당(약칭 수꼴당) (2) 친일매국당(약칭 친일당) (3) 극우적폐당(약칭 적폐당) 국민은 못모이게 하면서  지들은 모이네 미래통합당이라는 미래를 붙여놓고는 사리판단이 불투명한 노인들을 선동해서 등에 업고 있는 과거 구태들..


1
진짜의료진들 뭔고생이야 미친 교회 들아 코로나걸려도 병원찾지마 너희가 그렇게 기도하며 찬양하는 예수한테 치료해달라해 기도하면 다들어주신다며 기도해 코로나 낫게해달라고 전광훈과 미쳐날뛰는 태극기모욕단을 형사처벌해주세요~ 그들이 그토록 원하는 박정희시절처럼 쥐도새도 모르게 잡아가서 반병신 만들어주세요!


2
여름 되면 사라질 거라는 사람들 다 어디 갔나요 전XX 목사는 왜 기독교 이름으로 먹칠을 하는지 모르겠네요 전광훈이한테... 종교 자유고 내가 다니는 것도 자유지만 수칙을 제대로 지키지않고 확산이 되면 경제적으로 타격입는 사람들 더 많아지고 시간이 길어질수록 더 살기 힘들어질텐데...


3
황나땡은 나가 떨어져도 전광훈 태극기만 보면 어떤 당이 떠오르네;; 요즘 젊은 사람들은 일장기 들고 다니고 있나? 알만한사람들은 다 아는데 전목사를 모르는 사람들은 언론말만 믿고 진짜 사랑제일교회를 나쁘게 보겠지.


4
원조 신천지가 이 꼬라지니 신천지도 그 꼬라지지ㅋㅋㅋㅋ자식은 부모 따라가게 되어있음ㅋㅋㅋㅋ 코로나 제2의 확산 주범, 신천지 전광훈 청원 @t 전광훈이 신천지 2중대라니까.


5
전광훈 결국 일을 내는구나...국가 전복시키고 국민을 사지로몰고 경제를 파탄내는 자들이다.저자들은 또 공항을 안먹아서 실패한거니 정부 비판을하겠지 전세계 국가가 전부다 국경을 막고도 이지경인데 입만열면 정부부터 비판하니 이나라가 걱정이다.어찌 저런자가 나와서 국가를 전복시키려하고 정치에 개입해서 국민을 사지로몰고 나라를 이지경으로 만든단 말인가 테러집단도 치료해야 한다는 이 잣같은 현실이 개탄스럽다.


6
광복절날  일장기  흔들며   코로나  퍼뜨리는  사람들은  도대체  어느나라  국민인가요? 구상권청구~ 8.15광화문 시위

In [28]:
summary_channel_a_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   published_time              69 non-null     object 
 1   video_id                    69 non-null     object 
 2   video_url                   69 non-null     object 
 3   crawling_time               69 non-null     object 
 4   author                      69 non-null     object 
 5   title                       69 non-null     object 
 6   length                      69 non-null     float64
 7   rating                      69 non-null     float64
 8   views                       69 non-null     float64
 9   thumbnail_url               69 non-null     object 
 10  captions                    69 non-null     object 
 11  description                 69 non-null     object 
 12  comments                    69 non-null     object 
 13  cleaned_description         69 non-nu

In [29]:
summary_jtbc_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   published_time              149 non-null    object 
 1   video_id                    149 non-null    object 
 2   video_url                   149 non-null    object 
 3   crawling_time               149 non-null    object 
 4   author                      149 non-null    object 
 5   title                       149 non-null    object 
 6   length                      149 non-null    float64
 7   rating                      149 non-null    float64
 8   views                       149 non-null    float64
 9   thumbnail_url               149 non-null    object 
 10  captions                    149 non-null    object 
 11  description                 149 non-null    object 
 12  comments                    149 non-null    object 
 13  article_url                 149 non

In [30]:
summary_kbs_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   published_time              138 non-null    object 
 1   video_id                    138 non-null    object 
 2   video_url                   138 non-null    object 
 3   crawling_time               138 non-null    object 
 4   author                      138 non-null    object 
 5   title                       138 non-null    object 
 6   length                      138 non-null    float64
 7   rating                      138 non-null    float64
 8   views                       138 non-null    float64
 9   thumbnail_url               138 non-null    object 
 10  captions                    138 non-null    object 
 11  description                 138 non-null    object 
 12  comments                    138 non-null    object 
 13  cleaned_description         138 non

In [31]:
summary_mbc_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386 entries, 0 to 385
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   published_time              386 non-null    object 
 1   video_id                    386 non-null    object 
 2   video_url                   386 non-null    object 
 3   crawling_time               386 non-null    object 
 4   author                      386 non-null    object 
 5   title                       386 non-null    object 
 6   length                      386 non-null    float64
 7   rating                      386 non-null    float64
 8   views                       386 non-null    float64
 9   thumbnail_url               386 non-null    object 
 10  captions                    386 non-null    object 
 11  description                 386 non-null    object 
 12  comments                    386 non-null    object 
 13  article_url                 386 non

In [32]:
summary_mbn_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   published_time              161 non-null    object 
 1   video_id                    161 non-null    object 
 2   video_url                   161 non-null    object 
 3   crawling_time               161 non-null    object 
 4   author                      161 non-null    object 
 5   title                       161 non-null    object 
 6   length                      161 non-null    float64
 7   rating                      161 non-null    float64
 8   views                       161 non-null    float64
 9   thumbnail_url               161 non-null    object 
 10  captions                    161 non-null    object 
 11  description                 161 non-null    object 
 12  comments                    161 non-null    object 
 13  cleaned_description         161 non

In [33]:
summary_sbs_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   published_time              86 non-null     object 
 1   video_id                    86 non-null     object 
 2   video_url                   86 non-null     object 
 3   crawling_time               86 non-null     object 
 4   author                      86 non-null     object 
 5   title                       86 non-null     object 
 6   length                      86 non-null     float64
 7   rating                      86 non-null     float64
 8   views                       86 non-null     float64
 9   thumbnail_url               86 non-null     object 
 10  captions                    86 non-null     object 
 11  description                 86 non-null     object 
 12  comments                    86 non-null     object 
 13  article_url                 86 non-nu

In [34]:
summary_yonhab_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   published_time              472 non-null    object 
 1   video_id                    472 non-null    object 
 2   video_url                   472 non-null    object 
 3   crawling_time               472 non-null    object 
 4   author                      472 non-null    object 
 5   title                       472 non-null    object 
 6   length                      472 non-null    float64
 7   rating                      472 non-null    float64
 8   views                       472 non-null    float64
 9   thumbnail_url               472 non-null    object 
 10  captions                    472 non-null    object 
 11  description                 472 non-null    object 
 12  comments                    472 non-null    object 
 13  cleaned_description         472 non

In [35]:
summary_ytn_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 706 entries, 0 to 705
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   published_time              706 non-null    object 
 1   video_id                    706 non-null    object 
 2   video_url                   706 non-null    object 
 3   crawling_time               706 non-null    object 
 4   author                      706 non-null    object 
 5   title                       706 non-null    object 
 6   length                      706 non-null    float64
 7   rating                      706 non-null    float64
 8   views                       706 non-null    float64
 9   thumbnail_url               706 non-null    object 
 10  captions                    706 non-null    object 
 11  description                 706 non-null    object 
 12  comments                    706 non-null    object 
 13  cleaned_description         706 non

In [36]:
save_path = 'youtube_0816_0822/article_comments_summary/summary_{}_0816_0822.csv'

In [37]:
summary_channel_a_1.to_csv(save_path.format('channel_a'))
summary_jtbc_1.to_csv(save_path.format('jtbc'))
summary_kbs_1.to_csv(save_path.format('kbs'))
summary_mbc_1.to_csv(save_path.format('mbc'))
summary_mbn_1.to_csv(save_path.format('mbn'))
summary_sbs_1.to_csv(save_path.format('sbs'))
summary_yonhab_1.to_csv(save_path.format('yonhab'))
summary_ytn_1.to_csv(save_path.format('ytn'))

파일 불러올땐 pandas.read_csv()에서 index_col=0 옵션 추가해서 사용